In [1]:
#pip install unsloth transformers trl

In [2]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=2048,
    load_in_4bit=True
    )

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)


Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
tokenizer = get_chat_template(tokenizer, chat_template ='llama-3.1')

In [6]:
dataset=load_dataset("mlabonne/FineTome-100k",split="train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [7]:
dataset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [8]:
dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [9]:
dataset = dataset.map(
    lambda examples: {
    "text": [
        tokenizer.apply_chat_template(convo,tokenize=False)
        for convo in examples["conversations"]
    ]
    },
    batched=True
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [10]:
dataset

Dataset({
    features: ['conversations', 'source', 'score', 'text'],
    num_rows: 100000
})

In [12]:
trainer = SFTTrainer(
     model = model,
     train_dataset = dataset,
     dataset_text_field = "text",
     max_seq_length = 2048,
     args=TrainingArguments(
         per_device_train_batch_size = 2,
         gradient_accumulation_steps = 4,
         warmup_steps = 100,
         max_steps=60,
         learning_rate = 2e-4,
         fp16 = not torch.cuda.is_bf16_supported(),
         bf16=torch.cuda.is_bf16_supported(),
         logging_steps = 1,
         output_dir = "outputs"
     )
 )

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [13]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: scrapwebsites (scrapwebsites-ut-arlington-uta-the-university-of-texas-a) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.492200
2,2.629700
3,1.630000
4,2.066900
5,2.154700
6,2.359000
7,1.326700
8,2.509600
9,2.210000
10,1.819500


TrainOutput(global_step=60, training_loss=1.4903768529494603, metrics={'train_runtime': 530.5855, 'train_samples_per_second': 0.905, 'train_steps_per_second': 0.113, 'total_flos': 6240291994460160.0, 'train_loss': 1.4903768529494603})

In [14]:
model.save_pretrained("finetune_model")

In [15]:
inference_model, inference_tokenizer = FastLanguageModel.from_pretrained(
    model_name="./finetune_model",
    max_seq_length=2048,
    load_in_4bit=True
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [16]:
text_prompts=[
    "what are the key principles of investment?"
]

for prompt in text_prompts:
    formatted_prompt = inference_tokenizer.apply_chat_template([{
        "role": "user",
        "content": prompt
    }],tokenize=False)
    model_input = inference_tokenizer(formatted_prompt,return_tensors="pt").to("cuda")
    model_output = inference_model.generate(
        **model_input,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        pad_token_id=inference_tokenizer.pad_token_id,

    )
    repsonse = inference_tokenizer.decode(model_output[0],skip_special_tokens=True)
    print(repsonse)

system

Cutting Knowledge Date: December 2023
Today Date: 16 Apr 2025

user

what are the key principles of investment?assistant

The key principles of investment are:

1. Diversification: Spread investments across different asset classes, sectors, and geographic regions to minimize risk and maximize potential returns.
2. Risk management: Assess and manage investment risk through strategies such as hedging, diversification, and asset allocation.
3. Time horizon: Consider the investment time frame when making investment decisions, as it can impact the risk and potential returns of an investment.
4. Dollar-cost averaging: Invest a fixed amount of money at regular intervals, regardless of market conditions, to reduce the impact of market volatility.
5. Long-term focus: Focus on long-term investment goals, rather than short-term gains, as they can help to ride out market fluctuations and achieve more consistent returns.
6. Active management: Regularly review and adjust investment portfolio

In [17]:
text_prompts=[
    "what is the ideal calories taken per day?"
]

for prompt in text_prompts:
    formatted_prompt = inference_tokenizer.apply_chat_template([{
        "role": "user",
        "content": prompt
    }],tokenize=False)
    model_input = inference_tokenizer(formatted_prompt,return_tensors="pt").to("cuda")
    model_output = inference_model.generate(
        **model_input,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        pad_token_id=inference_tokenizer.pad_token_id,

    )
    repsonse = inference_tokenizer.decode(model_output[0],skip_special_tokens=True)
    print(repsonse)

system

Cutting Knowledge Date: December 2023
Today Date: 16 Apr 2025

user

what is the ideal calories taken per day?assistant

The ideal daily calorie intake varies based on factors such as age, sex, weight, height, activity level, and overall health goals. Here are some general guidelines:

1. For adult men:
   - Sedentary: 1,900-2,400 calories
   - Moderately active: 2,000-2,400 calories
   - Very active: 2,200-2,800 calories
   - Extremely active: 2,400-3,000 calories

2. For adult women:
   - Sedentary: 1,600-2,000 calories
   - Moderately active: 1,800-2,200 calories
   - Very active: 2,000-2,400 calories
   - Extremely active: 2,200-2,800 calories

3. For children and teenagers:
   - 2-3 years old: 1,000-1,400 calories
   - 4-8 years old: 1,200-1,600 calories
   - 9-13 years old: 1,400-2,000 calories
   - 14-18 years old: 1,600-2,200 calories

4. For pregnant or breastfeeding women:
   - 2,000-2,400 calories

5. For people with a higher body mass index (BMI):
   - 1,800-2,400 c

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
import random
import torch


base_model_name = "unsloth/Llama-3.2-3B-Instruct"
lora_model_path = "./finetune_model"


tokenizer = AutoTokenizer.from_pretrained(base_model_name)


base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16
).to("cuda")


model = PeftModel.from_pretrained(
    base_model,
    lora_model_path
).to("cuda")


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)


dataset = load_dataset("super_glue", "rte")
train_set = dataset["train"]
test_set = dataset["validation"]


def build_prompt(shot_type, test_sample, examples=None):
    if shot_type == "zero_shot":
        return f"Premise: {test_sample['premise']}\nHypothesis: {test_sample['hypothesis']}\n\nIs the hypothesis entailed by the premise? Answer with 'yes' or 'no'."

    elif shot_type == "one_shot" and examples:
        ex = examples[0]
        return f"""Example:
Premise: {ex['premise']}
Hypothesis: {ex['hypothesis']}
Answer: {"yes" if ex['label'] == 1 else "no"}

---

Now, determine the answer for:
Premise: {test_sample['premise']}
Hypothesis: {test_sample['hypothesis']}
Answer:"""

    elif shot_type == "few_shot" and examples:
        shots = ""
        for i, ex in enumerate(examples):
            answer = "yes" if ex['label'] == 1 else "no"
            shots += f"Example {i+1}:\nPremise: {ex['premise']}\nHypothesis: {ex['hypothesis']}\nAnswer: {answer}\n\n"
        shots += f"Now, determine the answer for:\nPremise: {test_sample['premise']}\nHypothesis: {test_sample['hypothesis']}\nAnswer:"
        return shots

    else:
        raise ValueError("Invalid shot type or missing examples.")

# === Evaluation Function ===
def evaluate_model(shot_type, train_set, test_set, n_shots=2, limit=5):
    predictions = []
    references = []

    for i in range(min(limit, len(test_set))):
        test_sample = test_set[i]

        if shot_type == "zero_shot":
            prompt = build_prompt(shot_type, test_sample)
        elif shot_type in ["one_shot", "few_shot"]:
            examples = random.sample(list(train_set), n_shots if shot_type == "few_shot" else 1)
            prompt = build_prompt(shot_type, test_sample, examples)
        else:
            raise ValueError("Unknown shot_type")

        output = pipe(
            prompt,
            max_new_tokens=5,
            temperature=0.7,
            top_p=0.9
        )[0]["generated_text"]

        # Clean prediction
        generated = output.replace(prompt, "").strip().split("\n")[0].lower()
        generated = "yes" if "yes" in generated else "no"

        predictions.append(generated)


        if 'label' in test_sample and test_sample['label'] in [0,1]:
            reference = "yes" if test_sample['label'] == 1 else "no"
        else:
            reference = "unknown"
        references.append(reference)

        print(f"\n👉 Sample {i+1} | Shot Type: {shot_type}")
        print(f"Prompt:\n{prompt}")
        print(f"Prediction:\n{generated}")
        print(f"Reference:\n{reference}")
        print("-" * 60)

    valid_pairs = [(p, r) for p, r in zip(predictions, references) if r != "unknown"]
    if valid_pairs:
        correct = sum(1 for p, r in valid_pairs if p == r)
        accuracy = correct / len(valid_pairs)
    else:
        accuracy = 0.0

    return {"accuracy": accuracy}


zero_shot_results = evaluate_model("zero_shot", train_set, test_set, limit=20)
one_shot_results = evaluate_model("one_shot", train_set, test_set, n_shots=1, limit=20)
few_shot_results = evaluate_model("few_shot", train_set, test_set, n_shots=3, limit=20)


print("\n✅ Evaluation Results:")
print(f"Zero-Shot Accuracy: {zero_shot_results['accuracy'] * 100:.2f}%")
print(f"One-Shot Accuracy: {one_shot_results['accuracy'] * 100:.2f}%")
print(f"Few-Shot Accuracy: {few_shot_results['accuracy'] * 100:.2f}%")


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeo


👉 Sample 1 | Shot Type: zero_shot
Prompt:
Premise: Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation.
Hypothesis: Christopher Reeve had an accident.

Is the hypothesis entailed by the premise? Answer with 'yes' or 'no'.
Prediction:
yes
Reference:
yes
------------------------------------------------------------

👉 Sample 2 | Shot Type: zero_shot
Prompt:
Premise: Yet, we now are discovering that antibiotics are losing their effectiveness against illness. Disease-causing bacteria are mutating faster than we can come up with new antibiotics to fight the new variations.
Hypothesis: Bacteria is winning the war against antibiotics.

Is the hypothesis entailed by the premise? Answer with 'yes' or 'no'.
Prediction:
no
Reference:
no
------------------------------------------------------------

👉 Sample 3 | Shot Type: zero_shot
Prompt:
Premise: Cairo is now home to some 15 million people - a burgeoning populatio